In [1]:
import tensorflow as tf
keras = tf.keras

2023-06-13 03:48:40.053173: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-13 03:48:40.943696: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
model=keras.models.load_model("/tmp/open_spiel_PVPNet_test_mlp.pb")

2023-06-11 00:30:47.411439: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-06-11 00:30:47.483468: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-06-11 00:30:47.483677: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [9]:
batch_size=30
environment_shape=(20,20,2)
state_shape=(1,)
E=tf.zeros([batch_size,*environment_shape])
S=tf.zeros([batch_size,*state_shape])
model.predict({"inputs_0":E,"inputs_1":S})

ValueError: in user code:

    File "/home/ramizouari/anaconda3/envs/StochasticGames/lib/python3.10/site-packages/keras/engine/training.py", line 2169, in predict_function  *
        return step_function(self, iterator)
    File "/home/ramizouari/anaconda3/envs/StochasticGames/lib/python3.10/site-packages/keras/engine/training.py", line 2155, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/ramizouari/anaconda3/envs/StochasticGames/lib/python3.10/site-packages/keras/engine/training.py", line 2143, in run_step  **
        outputs = model.predict_step(data)
    File "/home/ramizouari/anaconda3/envs/StochasticGames/lib/python3.10/site-packages/keras/engine/training.py", line 2111, in predict_step
        return self(x, training=False)
    File "/home/ramizouari/anaconda3/envs/StochasticGames/lib/python3.10/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from Non

In [10]:
model.

In [317]:
n=100
G=tf.cast(tf.random.uniform([n,n]) > 0,dtype=tf.float32)
W=tf.random.uniform([n,n]) * G
x=tf.random.uniform([n,1])
x=tf.stack([x,x])
G=tf.stack([G,G])
W=tf.stack([W,W])



import importlib
import mpg.gnn.gcn as gcn
import mpg.gnn.aggregator as agg
import mpg.gnn.sgcn as sgcn 
importlib.reload(gcn)
importlib.reload(agg)
importlib.reload(sgcn)
WGCN=gcn.WGCN

In [320]:
layer=WGCN(5,loops=False,normalization="unit")
layer2=WGCN(5,loops=False,normalization="mean")
Y=layer((G,W,x))
Z=layer2((G,W,Y))


In [321]:
Y

<tf.Tensor: shape=(2, 100, 5), dtype=float32, numpy=
array([[[ 2.364595 , -5.8917894, -1.6306539,  6.2325687, -6.030709 ],
        [ 2.3944468, -5.966171 , -1.6512402,  6.311252 , -6.106844 ],
        [ 2.3923035, -5.9608297, -1.649762 ,  6.3056026, -6.101377 ],
        [ 2.1069312, -5.2497764, -1.4529657,  5.553422 , -5.373558 ],
        [ 2.2156246, -5.5206046, -1.5279222,  5.839915 , -5.650772 ],
        [ 2.2426457, -5.5879326, -1.5465562,  5.9111366, -5.719687 ],
        [ 2.2155795, -5.520492 , -1.5278909,  5.8397956, -5.6506567],
        [ 2.3688703, -5.902442 , -1.6336021,  6.2438374, -6.0416126],
        [ 2.0590339, -5.130432 , -1.4199352,  5.427175 , -5.2514   ],
        [ 2.1686325, -5.403516 , -1.4955158,  5.7160535, -5.5309224],
        [ 2.5228539, -6.2861185, -1.7397912,  6.649706 , -6.434335 ],
        [ 2.2568812, -5.6234026, -1.5563732,  5.9486585, -5.755994 ],
        [ 2.388234 , -5.95069  , -1.6469556,  6.294876 , -6.090998 ],
        [ 2.3148189, -5.7677636, -1.5

[<tf.Variable 'TAGConv/kernel:0' shape=(1, 5, 2) dtype=float32, numpy=
 array([[[-0.04742214, -0.14084202],
         [-0.41123575,  0.13933949],
         [ 0.72605807,  0.03667752],
         [-1.2124313 ,  0.43771482],
         [-0.48633334,  0.4591117 ]]], dtype=float32)>]

In [163]:
aggregator1=agg.SumAggregation(loops=True,with_weights=False,normalization="unit")
def f(A,x):
    matrices=A
    matrices=matrices + tf.eye(num_rows=matrices.shape[-1],batch_shape=matrices.shape[:-2])
    degrees=tf.reduce_sum(matrices,axis=-1)
    D=tf.sqrt(tf.linalg.diag(degrees))
    return tf.linalg.matmul(tf.linalg.matmul(tf.linalg.pinv(D),tf.linalg.matmul(matrices,tf.linalg.pinv(D))),x)

data=tf.ones([2,5,1])

In [168]:
f(G,x)

<tf.Tensor: shape=(2, 5, 1), dtype=float32, numpy=
array([[[0.2452044 ],
        [0.6565305 ],
        [0.64284056],
        [0.4892034 ],
        [0.52406955]],

       [[0.2452044 ],
        [0.6565305 ],
        [0.64284056],
        [0.4892034 ],
        [0.52406955]]], dtype=float32)>

In [169]:
aggregator1((G,x))

<tf.Tensor: shape=(2, 5, 1), dtype=float32, numpy=
array([[[0.2452044 ],
        [0.6565305 ],
        [0.64284056],
        [0.48920345],
        [0.52406955]],

       [[0.2452044 ],
        [0.6565305 ],
        [0.64284056],
        [0.48920345],
        [0.52406955]]], dtype=float32)>

In [170]:
import numpy as np
P=np.zeros([2,5,1])
degree = tf.reduce_sum(G,axis=-1) +1 
for b in range(2):
    
    for i in range(5):
        P[b,i,0] = x[b,i,0] / degree[b,i]
        for j in range(5):
            P[b,i,0] += G[b,i,j]*x[b,j,0] / np.sqrt(degree[b,i] * degree[b,j])
P

array([[[0.2452044 ],
        [0.65653056],
        [0.64284056],
        [0.48920345],
        [0.52406955]],

       [[0.2452044 ],
        [0.65653056],
        [0.64284056],
        [0.48920345],
        [0.52406955]]])

In [167]:
matrices=G
matrices=matrices + tf.eye(num_rows=matrices.shape[-1],batch_shape=matrices.shape[:-2])
degrees=tf.reduce_sum(matrices,axis=-1,keepdims=True)
tf.linalg.matmul(matrices, data / tf.math.sqrt(degrees))/tf.math.sqrt(degrees)

<tf.Tensor: shape=(2, 5, 1), dtype=float32, numpy=
array([[[0.90824825],
        [1.        ],
        [1.1422286 ],
        [1.0749149 ],
        [1.        ]],

       [[0.90824825],
        [1.        ],
        [1.1422286 ],
        [1.0749149 ],
        [1.        ]]], dtype=float32)>

In [154]:
degrees.shape

TensorShape([2, 5])

In [528]:
import mpg.ml.dataset.reader as reader
import mpg.ml.dataset.generator as generator
import mpg.ml.dataset.utils as utils
importlib.reload(reader)
importlib.reload(generator)
importlib.reload(utils)
gen=reader.MPGGraphReader("/run/media/ramizouari/6cf4ec5b-ccaf-4b2a-a40f-75ddfe64af76/HPC/dataset/dense",target="all",target_path="/run/media/ramizouari/6cf4ec5b-ccaf-4b2a-a40f-75ddfe64af76/HPC/dataset/targets_dense.json")

(TensorSpec(shape=(2, None, None), dtype=tf.float32, name=None), TensorSpec(shape=(2, None), dtype=tf.float32, name=None))


AttributeError: '_OptionsDataset' object has no attribute 'target_data'

In [531]:
for x,y in gen:
    print(x)
    break

2023-06-13 08:32:13.330972: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-06-13 08:32:13.906668: W tensorflow/core/framework/op_kernel.cc:1818] INVALID_ARGUMENT: TypeError: `generator` yielded an element of shape (2, 2, 130) where an element of shape (2, None) was expected.
Traceback (most recent call last):

  File "/home/ramizouari/anaconda3/envs/StochasticGames/lib/python3.10/site-packages/tensorflow/python/ops/script_ops.py", line 267, in __call__
    ret = func(*args)

  File "/home/ramizouari/anaconda3/envs/StochasticGames/lib/python3.10/site-packages/tensorflow/python/autograph/impl/api.py", line 642, in wrapper
    return func(*args, **kwargs)

  File "/home/ramizouari/anaconda3/envs/StochasticGames/lib/python3.10/site-packag

InvalidArgumentError: {{function_node __wrapped__IteratorGetNext_output_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} TypeError: `generator` yielded an element of shape (2, 2, 130) where an element of shape (2, None) was expected.
Traceback (most recent call last):

  File "/home/ramizouari/anaconda3/envs/StochasticGames/lib/python3.10/site-packages/tensorflow/python/ops/script_ops.py", line 267, in __call__
    ret = func(*args)

  File "/home/ramizouari/anaconda3/envs/StochasticGames/lib/python3.10/site-packages/tensorflow/python/autograph/impl/api.py", line 642, in wrapper
    return func(*args, **kwargs)

  File "/home/ramizouari/anaconda3/envs/StochasticGames/lib/python3.10/site-packages/tensorflow/python/data/ops/from_generator_op.py", line 235, in generator_py_func
    raise TypeError(

TypeError: `generator` yielded an element of shape (2, 2, 130) where an element of shape (2, None) was expected.


	 [[{{node PyFunc}}]] [Op:IteratorGetNext]

2023-06-13 08:32:14.056054: W tensorflow/core/framework/op_kernel.cc:1818] INVALID_ARGUMENT: TypeError: `generator` yielded an element of shape (2, 2, 60) where an element of shape (2, None) was expected.
Traceback (most recent call last):

  File "/home/ramizouari/anaconda3/envs/StochasticGames/lib/python3.10/site-packages/tensorflow/python/ops/script_ops.py", line 267, in __call__
    ret = func(*args)

  File "/home/ramizouari/anaconda3/envs/StochasticGames/lib/python3.10/site-packages/tensorflow/python/autograph/impl/api.py", line 642, in wrapper
    return func(*args, **kwargs)

  File "/home/ramizouari/anaconda3/envs/StochasticGames/lib/python3.10/site-packages/tensorflow/python/data/ops/from_generator_op.py", line 235, in generator_py_func
    raise TypeError(

TypeError: `generator` yielded an element of shape (2, 2, 60) where an element of shape (2, None) was expected.


2023-06-13 08:32:14.056101: I tensorflow/core/common_runtime/executor.cc:1197] [/job:localhost/replica:0/ta

In [ ]:
for x in gen.bucket_by_sequence_length(lambda x:x.shape[-1],[20,50,500],[5,5,5,5],pad_to_bucket_boundary=True):
    z=x
    break

In [131]:
!protoc --version

libprotoc 3.19.6


In [ ]:
layer((z,z,z))

In [285]:
tf.linalg.det(tf.random.uniform([n,n]) 

SyntaxError: incomplete input (32705229.py, line 1)

In [440]:
import pandas as pd
df=pd.read_json("/run/media/ramizouari/6cf4ec5b-ccaf-4b2a-a40f-75ddfe64af76/HPC/dataset/targets_dense.json")
df.set_index("graph",inplace=True)


In [468]:
filename="gnp_uniform_mpg_300_30_108.edgelist.gz"

In [516]:
B=np.array(df.loc[filename,["winners_max","winners_min"]].to_list()).T
A=np.array(df.loc[filename,["max_strategy","min_strategy"]].to_list()).T
C=np.stack([B,A],axis=1)

In [523]:
C[:,1,0]

array([281, 116, 176, 194, 297, 287, 195, 148,  75, 262, 248, 158,  77,
       173,  14, 283, 118,   4, 102, 282, 191,  43, 191, 171, 245, 184,
       199, 125, 166,  11, 295,  46,  31, 159,  17, 245, 123, 185, 188,
       178, 154, 175, 185,  86, 287, 276, 175, 118,  52,  69, 163,  98,
       261, 261, 264, 264,  14, 274, 248, 242, 225, 213,  58, 234,  99,
        28, 209,  20,  89,  45, 196, 146,  38, 198, 141, 256,  63,  57,
       250, 262, 146, 228, 133, 136, 111,  20, 237,  66,  70,  65,  61,
       193, 130, 118, 256, 256, 168,  35, 185, 188, 122, 241,  21, 258,
       192, 256,   2, 160, 219,  86, 192, 297, 202, 217, 126, 290,  47,
        46, 135, 115,  67, 280,   8, 170, 143, 225, 186, 261,  85,   0,
        22, 115, 281,  50, 127, 195, 194,  45, 111,  86, 128, 252,  53,
       216, 133,  25, 121, 169, 213, 193, 227, 256,  53, 255,   6, 263,
        69, 163, 161, 147,  34,  65, 175, 147,  95, 197, 112, 252, 131,
       261,  90, 289, 144,  95,  91,  38, 172, 143,  82, 124, 21

(300, 2, 2)